# Kaggle - WiDS Datathon 2022

## Modelo Nº6

### Description:

- All the features
- Separation train/test - 80/20
- Catboost
- Hiperparameters by default
- Cross validation
- Long value of iterations: 10000

- Public Score = XXX

# 1. Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#model
from catboost import CatBoostRegressor

#train/test split
from sklearn.model_selection import train_test_split

#pool class
from catboost import Pool

#cross validation
from catboost import cv

# 2. Data

In [2]:
#paths to the Github repo

train_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/train.csv'
test_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/test.csv'
submit_path = 'https://raw.githubusercontent.com/ccollado7/wds-datathon-2022/main/data/raw/sample_solution.csv'

In [3]:
#train dataset

train = pd.read_csv(train_path)
train.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
#test dataset

test = pd.read_csv(test_path)
test.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,id
0,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75757
1,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75758
2,7,State_1,Commercial,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75759
3,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,38,50.596774,...,0,29,5,2,0,NaN,NaN,NaN,NaN,75760
4,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,38,50.596774,...,0,29,5,2,0,340.0,330.0,22.8,126.0,75761


In [5]:
#sumbit

submit = pd.read_csv(submit_path)
submit.head()

,id,site_eui
0,75757,0.0
1,75758,0.0
2,75759,0.0
3,75760,0.0
4,75761,0.0


In [6]:
x = train.drop(['id','site_eui'], axis=1)
x.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [7]:
y = train['site_eui']

In [8]:
cat_features = list(train.select_dtypes(include=['object']).columns)
cat_features

['State_Factor', 'building_class', 'facility_type']

In [10]:
#train / validation split
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2, random_state=0)

#train pool
train_pool = Pool(
    data=x_train, 
    label=y_train, 
    cat_features=cat_features
)

#validation pool
validation_pool = Pool(
    data=x_validation, 
    label=y_validation, 
    cat_features=cat_features
)

In [14]:
x_test = test.drop('id', axis=1)
x_test.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

## Feature Importance

In [23]:
model = CatBoostRegressor()
model.get_feature_importance(train_pool)

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/libs/fstr/calc_fstr.cpp:480: Model is not trained

### Model 6

In [20]:
model_6 = CatBoostRegressor(verbose=250,
                          iterations=10000,
                          early_stopping_rounds=50,
                          random_seed=0,
                          max_depth=12,
                          eval_metric='RMSE',
                          loss_function='RMSE'
)
model_6.fit(
    train_pool,
    eval_set=validation_pool,
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 57.8162381	test: 60.1623987	best: 60.1623987 (0)	total: 206ms	remaining: 34m 17s
250:	learn: 43.2716087	test: 47.0351848	best: 47.0351848 (250)	total: 51.9s	remaining: 33m 35s
500:	learn: 40.3642399	test: 46.0108426	best: 46.0108426 (500)	total: 1m 43s	remaining: 32m 48s
750:	learn: 38.3352204	test: 45.3939676	best: 45.3929352 (739)	total: 2m 37s	remaining: 32m 20s
1000:	learn: 37.1168585	test: 45.1079466	best: 45.1079466 (1000)	total: 3m 30s	remaining: 31m 33s
1250:	learn: 36.2070016	test: 44.9113483	best: 44.9113483 (1250)	total: 4m 25s	remaining: 30m 55s
1500:	learn: 35.3916881	test: 44.8368733	best: 44.8323653 (1489)	total: 5m 18s	remaining: 30m 1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 44.72647464
bestIteration = 1698

Shrink model to first 1699 iterations.


In [21]:
submit['site_eui'] = model_6.predict(x_test)
submit.to_csv('modelo_6.csv',index=False)

In [22]:
model_6.get_feature_importance

<bound method CatBoost.get_feature_importance of <catboost.core.CatBoostRegressor object at 0x00000151269CA850>>